In [1]:
from src.flight_analysis.scrape import Scrape

In [2]:
flights = Scrape("Frosinone", "Sydney", "2023-10-25")
flights.run_scrape()

In [3]:
flights.data

,departure_datetime,arrival_datetime,airlines,travel_time,origin,destination,layover_n,layover_time,layover_location,price_eur,price_trend,price_value,access_date,one_way,has_train,days_advance
0,2023-10-25 21:10:00,2023-10-27 13:35:00,[China Eastern],1 days 07:25:00,FCO,SYD,1,0 days 09:25:00,[PVG],694,low,194,2023-08-08 18:23:03.469385,True,False,78
1,2023-10-25 21:10:00,2023-10-27 10:00:00,[China Eastern],1 days 03:50:00,FCO,SYD,1,0 days 05:55:00,[PVG],924,low,194,2023-08-08 18:23:03.469406,True,False,78
2,2023-10-25 08:40:00,2023-10-26 21:30:00,"[ITA, THAI]",1 days 03:50:00,FCO,SYD,2,NaT,"[FRA, BKK]",961,low,194,2023-08-08 18:23:03.469420,True,False,77
3,2023-10-25 15:30:00,2023-10-26 23:05:00,"[Emirates, Qantas]",0 days 22:35:00,FCO,SYD,1,0 days 03:05:00,[DXB],1312,low,194,2023-08-08 18:23:03.469423,True,False,77
4,2023-10-25 10:20:00,2023-10-26 19:10:00,[Etihad],0 days 23:50:00,FCO,SYD,1,0 days 04:05:00,[AUH],1322,low,194,2023-08-08 18:23:03.469426,True,False,77
5,2023-10-25 08:40:00,2023-10-27 09:15:00,"[ITA, Vietnam Airlines]",1 days 15:35:00,FCO,SYD,2,NaT,"[CDG, SGN]",877,low,194,2023-08-08 18:23:03.469428,True,False,77
6,2023-10-25 08:40:00,2023-10-27 09:15:00,"[ITA, Vietnam Airlines]",1 days 15:35:00,FCO,SYD,2,NaT,"[FRA, SGN]",885,low,194,2023-08-08 18:23:03.469430,True,False,77
7,2023-10-25 15:30:00,2023-10-27 11:00:00,"[Turkish Airlines, Malaysia Airlines]",1 days 10:30:00,FCO,SYD,2,NaT,"[IST, KUL]",923,low,194,2023-08-08 18:23:03.469433,True,False,77
8,2023-10-25 10:45:00,2023-10-26 21:30:00,"[ITA, THAI]",1 days 01:45:00,FCO,SYD,2,NaT,"[ZRH, BKK]",1004,low,194,2023-08-08 18:23:03.469435,True,False,77
9,2023-10-25 10:20:00,2023-10-27 07:20:00,[Etihad],1 days 12:00:00,FCO,SYD,1,0 days 16:20:00,[AUH],1077,low,194,2023-08-08 18:23:03.469438,True,False,77


In [4]:
flights._url

'https://www.google.com/travel/flights?q=Flights%20to%20Sydney%20from%20Frosinone%20on%202023-10-25%20oneway&curr=EUR&gl=IT'

In [5]:
raise NotImplementedError()

NotImplementedError: 

In [ ]:
import pandas as pd
from src.flight_analysis.database import Database
import private.private as private
import numpy as np

db = Database(
    db_host=private.DB_HOST,
    db_name=private.DB_NAME,
    db_user=private.DB_USER,
    db_pw=private.DB_PW,
)

query = "SELECT uuid, layover_location FROM scraped"

cur = db.conn.cursor()
cur.execute(query)

res = cur.fetchall()
res

In [7]:
df = pd.DataFrame(res, columns=["flight_uuid", "layover_location"])
df

,flight_uuid,layover_location
0,bc9de66a-e11b-4ded-a230-0eccbc7f3838,None
1,37e1aaeb-80d9-4cd2-b24f-1d197a93671f,None
2,f6956f34-ddc4-4c57-80ef-706049058c34,None
3,8b0795c7-2acb-427e-81aa-4181827a483e,None
4,f7aa01e7-6fac-48ab-bb37-22f6b39c8213,None
...,...,...
111415,17893f94-0096-42d0-b71c-c5fedd3be13b,BCN
111416,a2dc98dd-6268-44ea-af71-598f6a59fad3,BCN
111417,dd722f79-10c7-4f1d-97fa-fc31b608c182,LHR
111418,aa93dd96-6ffd-4a69-ac9b-e464b7e24b43,AMS


In [8]:
# get rid of "min"
df.lauover_location = df.layover_location.str.strip()
df.layover_location = df.layover_location.replace("", np.nan)
df.layover_location = df.layover_location.replace("min", np.nan)
df.layover_location = df.layover_location.replace("Change of airport", np.nan)

df = df.dropna()
df

/var/folders/_6/c56rsvy15k9_gv99mkdll8pc0000gn/T/ipykernel_94061/3424689233.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.lauover_location = df.layover_location.str.strip()


,flight_uuid,layover_location
5,811962d6-e250-4480-bb8a-2d2ae511afe4,ATH
8,bea04f18-6fc3-4f39-8a57-caeaa25bf35b,LGW
9,941c0948-ceda-4a4f-bb60-f9fc5a322b95,BCN
19,4a86fa79-dc3d-4e92-9c03-412bdfbf3810,DUS
22,6a341fc6-9761-4cdc-bced-cf3fa636c2a9,BCN
...,...,...
111415,17893f94-0096-42d0-b71c-c5fedd3be13b,BCN
111416,a2dc98dd-6268-44ea-af71-598f6a59fad3,BCN
111417,dd722f79-10c7-4f1d-97fa-fc31b608c182,LHR
111418,aa93dd96-6ffd-4a69-ac9b-e464b7e24b43,AMS


In [9]:
db.create_scraped_layovers_table()

In [10]:
def split_dataframe(df, chunk_size=5000):
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i * chunk_size : (i + 1) * chunk_size])
    return chunks


chunks = split_dataframe(df, chunk_size=5000)

i = 1
for c in chunks:
    print(i, "/", len(chunks))
    db.add_pandas_df_to_db(c, table_name=db.table_scraped_layovers)
    i += 1

1 / 11
2 / 11
3 / 11
4 / 11
5 / 11
6 / 11
7 / 11
8 / 11
9 / 11
10 / 11
11 / 11


In [ ]:
# this process is to retroactively split the flight airlines into a separate "scraped_airlines" table,
# in order to violate the 1NF and 2NF rules of database normalization.

raise NotImplementedError("This script is not meant to be run again!!!!!")

import pandas as pd
from src.flight_analysis.database import Database
import private.private as private

# db = Database(
#     db_host=private.DB_HOST,
#     db_name=private.DB_NAME,
#     db_user=private.DB_USER,
#     db_pw=private.DB_PW,
# )

query = "SELECT id, airlines FROM scraped"

cur = db.conn.cursor()
cur.execute(query)

res = cur.fetchall()

df = pd.DataFrame(res, columns=["flight_uuid", "airline"])

df2 = df.explode("airline")
df2["airline"] = df2["airline"].map(lambda x: x.lstrip("'").rstrip("'"))
df2["airline"] = df2["airline"].replace(
    {"Separate tickets booked together": "multiple"}
)
df2 = df2.reset_index(drop=True)


def split_dataframe(df, chunk_size=5000):
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i * chunk_size : (i + 1) * chunk_size])
    return chunks


chunks = split_dataframe(df2, chunk_size=5000)

i = 1
for c in chunks:
    print(i, "/", len(chunks))
    db.add_pandas_df_to_db(c, table_name=db.table_scraped_airlines)
    i += 1

In [ ]:
import re
import numpy as np


def _is_arg_layover(arg):
    """ """
    # layover location cases
    layover_location_cases = {
        # case 1: (xx hr xx min AAA), (xx hr xx min Aaaaa)
        1: re.search("\d{0,2} hr \d{0,2} min [A-Z]+", arg),
        # case 2: (xx hr AAA), (xx hr AAA)
        2: re.search("\d{0,2} hr [A-Z]+", arg),
        # case 3: (xx min AAA), (xx min AAA)
        3: re.search("\d{0,2} min [A-Z]+", arg),
        # case 4: (AAA, BBB, ...)
        4: re.search("^[A-Z]{3}, ([A-Z]{3}(, )?)?", arg),
    }

    print(layover_location_cases)

    return any(layover_location_cases.values())


def _parse_layover_times_location(arg):
    layover_time = None
    layover_location = None

    # layover time
    if (" hr" in arg) or (" min" in arg):
        layover_time = (
            re.search("^(\d{1,2} hr){0,1}\s{0,1}(\d{1,2} min){0,1}\s", arg)
            .group()
            .strip()
        )
        layover_location = arg.split(" ")[-1]

    # layover location
    if "," in arg:
        layover_location = arg.split(", ")
        layover_location = [x.strip() for x in layover_location]

    return layover_time, layover_location


test_args = [
    "22 hr 12 min FCO",
    # "22 hr 12 min",
    # "3 min Ancona",
    # "13 min LAX",
    # "FCO, JFK",
    # "FCO, JFK",
    # "21 hr 5 min DOH",
    # "hehe",
    "SWISS, Singapore AirlinesOperated by Helvetic",
]

for arg in test_args:
    if _is_arg_layover(arg):
        x = _parse_layover_times_location(arg)
        print(arg, x)

In [6]:
import numpy as np

In [18]:
AIRPORTS_DF_URL = "https://raw.githubusercontent.com/datasets/airport-codes/master/data/airport-codes.csv"
df = pd.read_csv(AIRPORTS_DF_URL, encoding="utf-8")

# filter original data
# take only aiprort that have a IATA code
df = df.loc[df.iata_code.notnull()]

# exclude closed, seaplane_base and heliport
df = df.loc[df.type.isin(["large_airport", "medium_airport", "small_airport"])]

# split coordinates in latitude and longitude
df["lat"] = df.coordinates.apply(lambda x: x.split(",")[0]).astype(float)
df["lon"] = df.coordinates.apply(lambda x: x.split(",")[1]).astype(float)

# take only useful columns
cols = [
    "iata_code",
    "name",
    "iso_country",
    "lat",
    "lon",
    "iso_region",
    "municipality",
    "continent",
]
df = df[cols]

# rename columns
df = df.rename(
    columns={
        "iata_code": "iata",
        "iso_country": "country",
        "iso_region": "region",
    }
)

# replace nan with None
df = df.replace({pd.NA: None, np.nan: None})

df.loc[df.iata == "FCO"]

,iata,name,country,lat,lon,region,municipality,continent
40729,FCO,Rome–Fiumicino Leonardo da Vinci International...,IT,41.804532,12.251998,IT-62,Rome,EU


In [1]:
import pandas as pd
from src.flight_analysis.database import Database
import private.private as private

db = Database(
    db_host=private.DB_HOST,
    db_name=private.DB_NAME,
    db_user=private.DB_USER,
    db_pw=private.DB_PW,
)

In [2]:
db.list_all_tables()

['scraped', 'scraped_layovers', 'scraped_airlines']

In [3]:
db.create_data_airports_table()